# Parking Sign Recognition


### Steps

1. pictures of every possible sign [doesn't need words]
2. need to find python library to do the following a) Recognize the sign object b) match the sign with our database of base signs c) read the text

To Do step 2a, we are going to attempt to use opencv
First, pip install opencv-python


In [1]:
import cv2 #opencv is cv2

In [11]:
# Youtube tut: https://www.youtube.com/watch?v=T-0lZWYWE9Y&list=PLzMcBGfZo4-lUA8uGjeXhBUUzPYc6vZRn&index=7

# Loading Images
# imread has a second paramter of -1, 0 or 1
# -1: loads a colored image, neglecting transparency (default)
# 0: loads image in grayscale
# 1: includes image as it is, including transparency

imgt1 = cv2.imread('Parking Signs/Base Signs/no standing.jpg', 0) #imgt1 for image template1
img1 = cv2.imread('Parking Signs/Signs/cams house.jpg', 0) #img1, this is the image we will run template again.
img1 = cv2.resize(img1, (800,800)) # This image was way too big initially

# These next 3 lines displays the image
cv2.imshow('template1',imgt1) # you need the next two lines to run this
cv2.waitKey(0) # Waits a number of seconds until we press a key to move onto the next line, 0 indicates to wait infinitely
cv2.destroyAllWindows() # closes the image
cv2.imshow('image1',img1)
cv2.waitKey(0)
cv2.destroyAllWindows() 


# images are understood by python as numpy arrays representing 3 values per pixel [blue, green, red] where blue green red represent a number from 0-255
# the columns represent the width while the rows represent the height

In [15]:
# To recognize a sign, we can use corner detection (shi-thomasi Detector & goodFeaturesToTrack()). This may be problematic by picking up other corners in an image
# however this does allow you to specify a few paramaters to help improve accuracy. 
# Paramaters are: (input image, # of best corners to choose, confidence level, min euclidean distance between corners)

# Template matching is likely the algorithm to use. A potential issue is that the size of the template image needs to be close enough to the size in the actual image.
# In our case we are going to try and match pictures in Base Signs to picutres in Signs folder.
# This algorithm requires grayscale. Note that since we are using grayscale now instead of a 3 dimensional array [height, width, channel], its just [height, width]
# 6 different methods to template match. It is recommended to try each method and figure out which gets the best results.
# The methods are TM_CCOEFF, TM_CCOEFF_NORMED, TM_CCORR, TM_CCORR_NORMED, TM_SQDIFF, TM_SQDIFF_NORMED

methods = [cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR, cv2.TM_CCORR_NORMED, cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]

h, w = imgt1.shape

# loop through all methods and find the best one
for method in methods:
    img2 = img1.copy()
    
    result = cv2.matchTemplate(img2, imgt1, method) # this performs a convolution. In other words, it slides img2 all around imgt1 until it finds a reasonable match.
                                            # this returns a 2 dimension array telling us how confident of a match there is in each region of the image
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result) # we case about min_loc, max_loc. These are the locations with best match. Max or min is best, depending on the method.
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        location = min_loc
    else:
        location = max_loc

    bottom_right = (location[0] + w, location[1] + h)
    cv2.rectangle(img2, location, bottom_right, 255, 5) # We draw a rectangle at the location, 255 for black, line thickness 5
    cv2.imshow('Match', img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows() # Seems like method 1 is the only one that works for this example. Size of both images is very important. Need to figure out best sizing.